## __Cross-match with LAMOST by Radius Search__ ##

* __only support a *.csv with ra and dec list__  
 ( the result file is in the same folder as your csv file

In [1]:
# the necessary info
csv_dir = r'C:\Users\Administrator\Desktop\大创\Data\temp'
csv_name = r'selected_data_18_None'
search_distance_sec = 5.0 # radius [arcsec]
flag = True # Web Search Visualization: True is visible and False is not

In [2]:
# the needed module import
import pandas as pd   
import os
from tqdm import tqdm, trange   # not necessary, just a visible progress bar

In [3]:
csv_file = csv_dir + '/' + csv_name + '.csv'
num = sum(1 for line in open(csv_file))-1
source = pd.read_csv(csv_file , index_col = ['ra'])
Ra = list(source.T)
source = pd.read_csv(csv_file , index_col = ['dec'])
Dec = list(source.T)
source = ''

In [4]:
temp_txt = csv_dir + '/' + csv_name + '.txt'
work = open(temp_txt , mode='w')
for j in trange(0,num):
    work.write(str(Ra[j]) + ',' + str(Dec[j]) + ',' + str(search_distance_sec) + '\n')
work.close()

100%|██████████| 336066/336066 [00:01<00:00, 231631.47it/s]


In [5]:
# the needed module import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import time

import urllib as url
import os

import re

In [6]:
#displayed or not setting
opt = Options()
if flag is not True:
    opt.add_argument('--headless')
driver = webdriver.Edge(options = opt)

In [7]:
driver.get('http://www.lamost.org/dr8/v2.0/search')
#上传
driver.find_element(By.XPATH,"//html/body/div[2]/form/div[2]/div[1]/div/table/tbody/tr[4]/td[3]/input").send_keys(temp_txt)
#选择 proximity 查找
driver.find_element(By.XPATH,"//html/body/div[2]/form/div[2]/div[1]/div/table/tbody/tr[4]/td[1]/span/input").click()
#点击 search
driver.find_element(By.XPATH,"//html/body/div[2]/form/div[2]/div[2]/input[1]").click()
#HTML进入跳转页面并等待3s，防止页面未显示
driver.switch_to.window(driver.window_handles[-1])
time.sleep(3)

In [8]:
#获取sqlid，以便导出匹配结果
query = url.parse.urlparse(driver.current_url).query
sqlid = re.findall("\d+",query)[0]
driver.close()

In [9]:
#匹配结果下载
response = url.request.urlopen('http://www.lamost.org/dr8/v2.0/sqlidall/{0}?output.fmt=csv&'.format(sqlid))
name = response.getheader("Content-disposition").split('=')[1]
data = response.read()
savefile = csv_dir + '/' + name
with open(savefile , 'wb+') as fh:
    fh.write(data)
    fh.close()
result_file = csv_dir + '/' + csv_name + '_cross_match.csv'
os.rename(savefile , result_file)